In [6]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [4]:
# Libraries
%load_ext autoreload
%autoreload 2
import index_toolkit as tk
import pandas as pd
import yfinance as yf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Inputs
date_input = '13/09/2024'
df_market_input = '^FCHI'
riskfree_input = '^TNX'
df_allocation = pd.read_csv('index_allocation.csv', index_col = 'Ticker')

In [6]:
# Fetching df_market data and risk-free rate
df_market = yf.Ticker(df_market_input).history('max')['Close']
df_market.name = df_market_input
df_market.index = df_market.index.tz_localize(None)
riskfree = yf.Ticker(riskfree_input).history('1y')['Close'].iloc[-1] / 100

# Fetching and reating a DataFrame to store historical prices for each asset
df_history = pd.DataFrame()
for ticker in df_allocation.index:
    history = yf.Ticker(ticker).history('max')['Close']
    history.name = ticker
    df_history = pd.concat([df_history, history], axis=1)
df_history.index = pd.to_datetime(df_history.index)
df_history = df_history.resample('B').last().ffill()
df_history.index = df_history.index.tz_localize(None)

In [ ]:
index_history = pd.read_csv('index_history.csv', index_col = 'Ticker')
index_history.Date = pd.to_datetime(index_history.Date)

# Updating portfolio history if it's a new day
if date_input not in index_history['Date'].to_list():
    index_history = pd.concat([index_history, dated_index])
    index_history = index_history.sort_values('Date')
    index_history.to_csv('portfolio_history.csv')
elif date_input not in index_history.Date.to_list():
    index_history = index_history.reset_index().set_index('Date').drop(index = today)
    index_history = index_history.reset_index().set_index('Ticker')
    index_history = pd.concat([index_history, dated_index])
    index_history = index_history.sort_values('Date')
    index_history.to_csv('portfolio_history.csv')

In [13]:
get_df_index = tk.index_table(
    history=df_history,
    allocation=df_allocation,
    date=date_input
)

df_index = get_df_index.compute()

get_df_index.show()

               Price   Share  Valorisation Allocation   Type  Account
Ticker                                                               
ACA.PA          14.2   815.0    11,601.5     27.36%    STOCKS    PEE 
WPEA.PA          5.2 1,750.0     9,135.5     21.54%       ETF    PEA 
UST.PA          71.1    60.0     4,264.1     10.06%       ETF    CTO 
AEME.PA         64.2    35.0     2,248.2      5.30%       ETF    CTO 
MC.PA          608.1     4.0     2,432.4      5.74%    STOCKS    PEA 
SAN.PA         103.4    15.0     1,551.0      3.66%    STOCKS    PEA 
STLAP.PA        13.6   100.0     1,361.0      3.21%    STOCKS    PEA 
AIR.PA         130.5    10.0     1,305.0      3.08%    STOCKS    PEA 
TTE.PA          61.1    20.0     1,221.4      2.88%    STOCKS    PEA 
DG.PA          109.5    10.0     1,095.0      2.58%    STOCKS    PEA 
AI.PA          170.2    10.0     1,702.2      4.01%    STOCKS    PEA 
SU.PA          227.2     5.0     1,136.0      2.68%    STOCKS    PEA 
0P00018AOF.F   183.6

In [10]:
df_index = tk.index_table(df_allocation, df_history, date_input)

df_display = df_index.copy()
df_display = df_index.drop(columns=['Date'])
df_display['Allocation'] = df_display['Allocation'].apply(lambda x: f"{x * 100:.2f}%")

with pd.option_context(
    'display.max_rows', None,
    'display.max_columns', None,
    'display.float_format', '{:,.1f}'.format,
    'display.max_colwidth', None,
    'display.colheader_justify', 'center'
):
    print(df_display)

               Price   Share  Valorisation Allocation   Type  Account
Ticker                                                               
ACA.PA          14.2   815.0    11,601.5     27.36%    STOCKS    PEE 
WPEA.PA          5.2 1,750.0     9,135.5     21.54%       ETF    PEA 
UST.PA          71.1    60.0     4,264.1     10.06%       ETF    CTO 
AEME.PA         64.2    35.0     2,248.2      5.30%       ETF    CTO 
MC.PA          608.1     4.0     2,432.4      5.74%    STOCKS    PEA 
SAN.PA         103.4    15.0     1,551.0      3.66%    STOCKS    PEA 
STLAP.PA        13.6   100.0     1,361.0      3.21%    STOCKS    PEA 
AIR.PA         130.5    10.0     1,305.0      3.08%    STOCKS    PEA 
TTE.PA          61.1    20.0     1,221.4      2.88%    STOCKS    PEA 
DG.PA          109.5    10.0     1,095.0      2.58%    STOCKS    PEA 
AI.PA          170.2    10.0     1,702.2      4.01%    STOCKS    PEA 
SU.PA          227.2     5.0     1,136.0      2.68%    STOCKS    PEA 
0P00018AOF.F   183.6

In [11]:
# Statistiques du portefeuille par positions
tk.stats_table(
    history=df_history, 
    market=df_market, 
    riskfree=riskfree, 
    allocation=df_index.Allocation, 
    date=date_input
).show()

interactive(children=(Select(description='freq', options=('D', 'B', 'W', 'M'), value='D'), Select(description=…

In [12]:
tk.chart_comparison(
    history=df_history, 
    date=date_input
).show()

interactive(children=(Select(description='freq', options=('D', 'B', 'W', 'M'), value='D'), Select(description=…

In [13]:
tk.top_n_flop(
    history=df_history, 
    date=date_input
).show()

interactive(children=(Select(description='range', options=('1W', '1M', 'YTD', '1Y', '5Y', '10Y'), value='1W'),…